<style>
div.blue{
    background-color:#e6f0ff; 
    border-radius: 5px; 
    padding: 20px;}
</style> 

<style>
div.warn {    
    background-color: #fcf2f2;
    border-color: #dFb5b4;
    border-left: 5px solid #dfb5b4;
    padding: 0.5em;
    }
 </style>
    
<h1 style="text-align: center; color: purple;" markdown="1">Econ 320 Python: Heteroscedasticity </h1>
<h2 style="text-align: center; color: purple;" markdown="1">Handout </h2>

 


#### Package setup

In [34]:
import wooldridge as woo
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf 
# Important new packages for this topic
import statsmodels.stats.api as sms
import statsmodels.api as sm
import patsy as pt

from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

# Heteroscedasticity

The homoscedasticity assumptions require that the variance of the error terms is unrelated to the regressors, i.e. 

$$Var(u|x_1, ..., x_k)=\sigma^2.$$ 

If homoscedasticity is violated, the standard errors are invalid and all inferences from t, F and other tests based on them are unreliable. In that case, we are facing heteroscedasticity in the standard errors: the circumstance in which the variability of the standard errors is unequal across the range of values of the predicted value of the dependent variable.

## Heteroscedasticity-Robust Inference


As you learned in class you need to modify the standard errors to correct for heteroscedasticity. This is having "heteroscedasticity-robust standard errors." 

In Python, we can do this using the `statsmodels` package. Using the  argument `cov_type` in the method `.fit()` we can obtain regression results that produced several refined versions of the white formula presented in your book. (Wooldridge, 2019). 

Let's say that the results from your regression are stored in the object *reg.*, then the variance- covariance matrix can be calculated using 

* **reg.fit(cov_type='nonrobust')** or **reg.fit()** for the default homoscedasticy-based standard errors. 
* **reg.fit(cov_type='HC0')**  for the classical version of White's robust varinace-covariance matrix by Wooldridge (2019, Equation 8.4 in Section 8,2)
* **reg.fit(cov_type='HC1')**  for the classical version of White's robust varinace-covariance matrix corrected by degrees of freedom.  
* **reg.fit(cov_type='HC2')**  for a version with small sample correction. This isndefault behavious of Stata. 
* **reg.fit(cov_type='HC3')**  for the refined version of version White's robust variance- covariance matrix. 

### Example with GPA data from wooldridge

For the spring semester run the regression 

$$ Cumulative GPA = \beta_0 + \beta_1SAT + \beta_2HSpercentile + \beta_3Totalhours + \beta_4 female + \beta_5 black + \beta_6 white $$


In [35]:
gpa3 = woo.dataWoo('gpa3')

# define regression model:
reg = smf.ols(formula='cumgpa ~ sat + hsperc + tothrs + female + black + white',
              data=gpa3, subset=(gpa3['spring'] == 1))
results_default = reg.fit()

# estimate default model (only for spring data):

results_default.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 cumgpa   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     39.98
Date:                Fri, 18 Apr 2025   Prob (F-statistic):           3.41e-37
Time:                        13:38:13   Log-Likelihood:                -238.90
No. Observations:                 366   AIC:                             491.8
Df Residuals:                     359   BIC:                             519.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4701      0.230      6.397      0.000       1.018       1.922
sat            0.0011      0.000      6.389      0.000       0.001       0.001
hsperc        -0.0086      0.001     -6.906      0.000      -0.011      -0.006
tothrs         0.0025      0.001      3.426      0.001       0.001       0.004
female         0.3034      0.059      5.141      0.000       0.187       0.420
black         -0.1283      0.147     -0.870      0.385      -0.418       0.162
white         -0.0587      0.141     -0.416      0.677      -0.336       0.219
==============================================================================
Omnibus:                        5.271   Durbin-Watson:                   1.865
Prob(Omnibus):                  0.072   Jarque-Bera (JB):                6.429
Skew:                          -0.123   Prob(JB):                       0.0402
Kurtosis:                       3.601   Cond. No.                     1.01e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [36]:
# estimate model with White SE (only for spring data):
results_white = reg.fit()
results_default.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 cumgpa   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     39.98
Date:                Fri, 18 Apr 2025   Prob (F-statistic):           3.41e-37
Time:                        13:38:13   Log-Likelihood:                -238.90
No. Observations:                 366   AIC:                             491.8
Df Residuals:                     359   BIC:                             519.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4701      0.230      6.397      0.000       1.018       1.922
sat            0.0011      0.000      6.389      0.000       0.001       0.001
hsperc        -0.0086      0.001     -6.906      0.000      -0.011      -0.006
tothrs         0.0025      0.001      3.426      0.001       0.001       0.004
female         0.3034      0.059      5.141      0.000       0.187       0.420
black         -0.1283      0.147     -0.870      0.385      -0.418       0.162
white         -0.0587      0.141     -0.416      0.677      -0.336       0.219
==============================================================================
Omnibus:                        5.271   Durbin-Watson:                   1.865
Prob(Omnibus):                  0.072   Jarque-Bera (JB):                6.429
Skew:                          -0.123   Prob(JB):                       0.0402
Kurtosis:                       3.601   Cond. No.                     1.01e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
# estimate model with refined White SE (only for spring data):
results_refined = reg.fit(cov_type='HC3')

#pay attention to the different standard errors the betas do not change
#with robust correction
variable_names = results_refined.params.index.tolist()
print(variable_names)

['Intercept', 'sat', 'hsperc', 'tothrs', 'female', 'black', 'white']


## Inference 
### T-test  

We are putting toguether the three models the results assuming homoskedasticity and assuming heteroskedasticity. ( Yes it can also be written heteroscedasticity) in a stargazer table. See that the coefficnets remain the same, the standard errors change. In this case we still reject the null hypothesis, this is not always the case. 



In [38]:
# put all results in stargazer table and 
models = Stargazer([results_default, results_white, results_refined])
models.covariate_order(['Intercept', 'sat', 'hsperc', 'tothrs', 'female', 'black', 'white'])
HTML(models.render_html())

### The F- test 

For the joint significance of all the coefficients you can check results for the F test on the regression table or  stargazer table. 

For other joint hypothesis like 

$$H_0: \beta_5 = 0 \: \&  \: \beta_6=0$$

See examples below. 
 

In [39]:
# definition of model and hypotheses:
reg = smf.ols(formula='cumgpa ~ sat + hsperc + tothrs + female + black + white',
              data=gpa3, subset=(gpa3['spring'] == 1))
hypotheses = ['black = 0', 'white = 0']

# F-Tests using different variance-covariance formulas:
# ususal VCOV:
results_default = reg.fit()
ftest_default = results_default.f_test(hypotheses)
fstat_default = ftest_default.statistic
fpval_default = ftest_default.pvalue
print(f'fstat_default: {fstat_default}\n')
print(f'fpval_default: {fpval_default}\n')

# refined White VCOV:
results_hc3 = reg.fit(cov_type='HC3')
ftest_hc3 = results_hc3.f_test(hypotheses)
fstat_hc3 = ftest_hc3.statistic
fpval_hc3 = ftest_hc3.pvalue
print(f'fstat_hc3: {fstat_hc3}\n')
print(f'fpval_hc3: {fpval_hc3}\n')

# classical White VCOV:
results_hc0 = reg.fit(cov_type='HC0')
ftest_hc0 = results_hc0.f_test(hypotheses)
fstat_hc0 = ftest_hc0.statistic
fpval_hc0 = ftest_hc0.pvalue
print(f'fstat_hc0: {fstat_hc0}\n')
print(f'fpval_hc0: {fpval_hc0}\n')

fstat_default: 0.679604195607341

fpval_default: 0.5074683622584049

fstat_hc3: 0.6724692957656635

fpval_hc3: 0.5110883633440992

fstat_hc0: 0.7477969818036214

fpval_hc0: 0.4741442714738484



## Correct for Heteroscedasticity in Homework Exercice
### 1
Run the model for the `sleep75` data set as appears in the book and test if the variance depends on the gender variable. 

In [40]:
#importing Card 1995 data: 
card = pd.read_stata('card.dta')
def dummy(x):
    if x==1:
        return 1
    else: 
        return 0
card['married_d']=card["married"].apply(dummy)
#df, meta = prs.read_dta('card.dta', apply_value_formats=True)

s1= smf.ols(formula='np.log(wage) ~ educ + exper + expersq + married_d + black', data=card).fit()
s2= smf.ols(formula='np.log(wage) ~ educ + exper + expersq + married_d + black', 
            data=card).fit(cov_type = 'HC3')

models = Stargazer([s1, s2])
models.title('Table 1: Regression on Wages and Heterocedasticity Correction ')
models.custom_columns(['Model 1', 'Model 2 '], [1, 1])
models.covariate_order(['Intercept', 'educ' , 'exper' , 'expersq', 'married_d','black'])
HTML(models.render_html())

### 2 
Use the data in `HPRICE1` to obtain the heteroskedasticity-robust standard errors for equation

$$price = \beta_0 + \beta_1*lotsize + \beta_2*sqrft + \beta_3*bdrms $$ 

Discuss any important differences with the usual standard errors. 
(ii)	Repeat part (i) for equation 
$$log(price) = log(lotsize) + log(sqrft) + bdrms$$.
(iii)	What does this example suggest about heteroskedasticity and the transformation used for the dependent variable?

**Take the log-transform of the dependent variable
Taking the log transform of the dependent variable is one of the most commonly used techniques for not only linearizing the dependent variable y but for also 'dampening down' the heteroscedastic variance (if it exists) in y.**


In [41]:
hprice1 = woo.dataWoo('hprice1')
results=smf.ols('price ~ lotsize + sqrft + bdrms', data=hprice1).fit()
# Refined White heteroscedasticity-robust SE:
results_rob=smf.ols('price ~ lotsize + sqrft + bdrms', data=hprice1).fit(cov_type='HC3')


In [42]:
resultslog=smf.ols("np.log(price)~np.log(lotsize)+np.log(sqrft)+bdrms", data=hprice1).fit()
resultslog_rob=smf.ols("np.log(price)~np.log(lotsize)+np.log(sqrft)+bdrms", 
                       data=hprice1).fit(cov_type='HC3')

# Refined White heteroscedasticity-robust SE:
# put all results in stargazer table and 
modelslog = Stargazer([resultslog, resultslog_rob,resultslog,results_rob])
#models.covariate_order(['Intercept','female' , 'educ' , 'exper', 'tenure', 'married',])
HTML(modelslog.render_html())

In [43]:
!jupyter nbconvert --to html H12E_320_HeteroscedasticityS24.ipynb

[NbConvertApp] Converting notebook H12E_320_HeteroscedasticityS24.ipynb to html
[NbConvertApp] Writing 304036 bytes to H12E_320_HeteroscedasticityS24.html


&nbsp;
<hr />
<p style="font-family:palatino; text-align: center;font-size: 15px">ECON320 Python Programming Laboratory</a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px">Professor <em> Paloma Lopez de mesa Moyano</em></a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px"><span style="color: #6666FF;"><em>paloma.moyano@emory.edu</em></span></p>

<p style="font-family:palatino; text-align: center;font-size: 15px">Department of Economics</a></p>
<p style="font-family:palatino; text-align: center; color: #012169;font-size: 15px">Emory University</a></p>

&nbsp;